In [22]:
from pymongo import MongoClient

client = MongoClient('mongodb://203.255.92.141:27017', authSource='admin')
filter_info = client['PUBLIC']['FilterInfo'] #필터접근

f_id = 1 #input
nyear = []
pyear = []

fid_key_query = filter_info.find_one({'fId' : f_id }) #필터검색

if  f_id > 0 : #필터쿼리가 있으면
    for key in fid_key_query.keys() :
        if key ==  'nFilter':
            nyear = fid_key_query[key]['year']
            #print(nyear)
        elif key == 'pFilter' :
            pyear = fid_key_query[key]['year']
            #print(pyear)
else : #필터쿼리가 없으면
    print("통합될 아이디가 없습니다")

keyid = fid_key_query['keyId'] #keyid

scion_aut = client['SCIENCEON']['Author']
ntis_aut = client['NTIS']['Author']
dbpia_aut = client['DBPIA']['Author']

scion_autpaper = client['SCIENCEON']['AuthorPapers']
ntis_autpaper = client['NTIS']['AuthorPapers']
dbpia_autpaper = client['DBPIA']['AuthorPapers']

scion_raw = client['SCIENCEON']['Rawdata']
ntis_raw = client['NTIS']['Rawdata']
dbpia_raw = client['DBPIA']['Rawdata']

scion_key_query = scion_autpaper.find({ 'keyId' : keyid })
ntis_key_query = ntis_autpaper.find({ 'keyId' : keyid })
dbpia_key_query = dbpia_autpaper.find({ 'keyId' : keyid })

auts = [scion_aut, ntis_aut, dbpia_aut] #저자 이름, 소속
key_querys = [scion_key_query, ntis_key_query, dbpia_key_query] #a_id

a_id = []
all_name = []
all_inst = []
all_site = []
Answer_dict = {} #통합결과
Answer_pdict = {}

site = ['Scienceon', 'NTIS', 'DBPIA']

for i in range(len(key_querys)):
    for key_query in key_querys[i]: #keyid에 저자수만큼 반복
        Aid = []
        A_id = []
        all_paper = []

        a_id.append(key_query['A_ID']) #a_id read

        if site[i] == 'NTIS' : 
            ntis_raw_query = ntis_raw.find({'$and':[{'keyId':keyid},{'mngId':a_id[-1]}]})
            if ntis_raw_query == None:
                continue
            else:
                for raw_one in ntis_raw_query:
                    if nyear[0] >= int(raw_one['prdEnd'][:4]) and nyear[1] <= int(raw_one['prdStart'][:4]) : #필터링
                        Aid = key_query['A_ID']
                        all_paper = key_query['papers']

        elif site[i] == 'Scienceon' :
            scion_raw_query = scion_raw.find({'$and':[{'keyId':keyid},{'author_id':{'$regex':a_id[-1]}}]})
            
            if scion_raw_query == None :
                continue
            else:
                for raw_one in scion_raw_query:
                    if pyear[0] >= int(raw_one['issue_year'][:4]) and pyear[1] <= int(raw_one['issue_year'][:4]): #필터링
                        A_id.append(key_query['A_ID'])
                        for v in A_id:
                            if v not in Aid:
                                Aid.append(v)
                                all_paper = key_query['papers']

        elif site[i] == 'DBPIA' :
            dbpia_raw_query = dbpia_raw.find({'$and':[{'keyId':keyid},{'author_id':{'$regex':a_id[-1]}}]})
            if dbpia_raw_query == None :
                continue
            else:
                for raw_one in dbpia_raw_query:
                    if pyear[0] >= int(raw_one['issue_year'][:4]) and pyear[1] <= int(raw_one['issue_year'][:4]): #필터링
                        A_id.append(key_query['A_ID'])
                        for v in A_id:
                            if v not in Aid:
                                Aid.append(v)
                                all_paper = key_query['papers']
                                
        if Aid == [] or Aid == "":
            continue

        aut_query = auts[i].find_one({'_id':key_query['A_ID']}) #저자이름, 소속 read

        all_name.append(aut_query['name'])
        all_inst.append(aut_query['inst'].replace("(주) ", "").replace("(주)", "").split(' ')[0])
        all_site.append(site[i])
        
        Answer = {'name' : all_name[-1], site[i] : {'inst' : all_inst[-1], 'A_id': Aid} }
        Answer_paper = {'name' : all_name[-1], site[i] : all_paper}

        if all_name[-1] not in Answer_dict and all_name[-1]+'0' not in Answer_dict :
            Answer_dict[all_name[-1]] = Answer
            Answer_pdict[all_name[-1]] = Answer_paper
        else :
            
            count = 0
            flag = True
            while flag :
                temp = None 
                tempName = all_name[-1]
              
                if tempName in Answer_dict :        # 이름 으로만 key가ㅣ 존재         
                    temp = Answer_dict[tempName]
                    flag = False
                else :
                    tempName = all_name[-1]+str(count)  # 이름 + 숫자로 key가ㅣ 존재
                    if tempName not in Answer_dict :
                        flag = False 
                        break
                    temp = Answer_dict[tempName]
                      
                for key in temp.keys() : # 사이트 돌면서
                    if key != 'name' : 
                        src = ""
                        tgt = ""

                        if len(all_inst[-1]) >= len(temp[key]['inst']):
                            src = temp[key]['inst']
                            tgt = all_inst[-1]

                        elif len(all_inst[-1]) < len(temp[key]['inst']):
                            src = all_inst[-1]
                            tgt = temp[key]['inst']
                        #print(temp)
                        if key == site[i] :# 사이트가 동일할때
                            if temp[key]['inst'] == all_inst[-1] or (src != "" and src in tgt) :  # 소속 같을때
                                flag = False
                                break

                            elif all_name[-1]+str(count+1) not in Answer_dict : #소속이 다를 때
                                Answer_dict[all_name[-1]+str(count+1)] = Answer
                                Answer_pdict[all_name[-1]+str(count+1)] = Answer_paper

                                if tempName == all_name[-1]:
                                    Answer_dict[all_name[-1]+'0'] = temp
                                    Answer_pdict[all_name[-1]+'0'] = Answer_pdict[all_name[-1]]
                                    del Answer_dict[all_name[-1]]
                                    del Answer_pdict[all_name[-1]]

                        else :# 사이트가 다를때 
                            if temp[key]['inst'] == all_inst[-1] or (src != "" and src in tgt):  # 소속 같을때
                                    Answer_dict[tempName][site[i]] =  {'inst' : all_inst[-1], 'A_id': Aid}
                                    Answer_pdict[tempName][site[i]] = all_paper
                                    flag = False
                                    break
                            
                            elif all_name[-1]+str(count+1) not in Answer_dict :
                                Answer_dict[all_name[-1]+str(count+1)] = Answer
                                Answer_pdict[all_name[-1]+str(count+1)] = Answer_paper

                                if tempName == all_name[-1]:
                                    Answer_dict[all_name[-1]+'0'] = temp
                                    Answer_pdict[all_name[-1]+'0'] = Answer_pdict[all_name[-1]]
                                    del Answer_dict[all_name[-1]]
                                    del Answer_pdict[all_name[-1]]  
                count += 1

#print(len(a_id), len(all_name), len(all_inst), len(Answer_dict.items()))
#print(Answer_dict)
#print(sorted(Answer_dict.items()))
#print(Answer_pdict)
id_domestic = client['ID']['Domestic']

for Answer_one in Answer_dict.keys() :
    input_id = "{'fid': " + str(f_id) + ", 'keyId': " + str(keyid)
    
    if len(Answer_dict[Answer_one].keys()) == 2: #site 2개 이상이 아니면 input안함
        continue
    
    for key in Answer_dict[Answer_one].keys() :
        if key == 'Scienceon' :
            input_id +=  ", '" + key + "': " + str(Answer_dict[Answer_one][key]['A_id'])
        elif key == "NTIS" :
            input_id +=  ", '" + key + "': '" + Answer_dict[Answer_one][key]['A_id'] + "'"
        elif key == "DBPIA" :
            input_id +=  ", '" + key + "': " + str(Answer_dict[Answer_one][key]['A_id'])
    input_id += "}"

for Answerp_one in Answer_pdict.keys() :
    #print(Answer_pdict[Answerp_one].keys())
    if len(Answer_pdict[Answerp_one].keys()) == 2: #site 2개 이상이 아니면 input안함
        continue
    else:
        print(Answer_pdict[Answerp_one])
    #print(eval(input_id))

    #id_domestic.insert_one(eval(input_id))


{'name': '최도진', 'Scienceon': [ObjectId('61939443e9b04a9d64abe055'), ObjectId('61939447e9b04a9d64abe06a')], 'DBPIA': [ObjectId('6193945c05488d4887ad9484')]}
{'name': '박수빈', 'Scienceon': [ObjectId('61939443e9b04a9d64abe055'), ObjectId('61939447e9b04a9d64abe06a')], 'DBPIA': [ObjectId('6193945205488d4887ad9318')]}
{'name': '조재희', 'Scienceon': [ObjectId('61939447e9b04a9d64abe066')], 'DBPIA': [ObjectId('6193945605488d4887ad9392')]}
{'name': '황지영', 'NTIS': [ObjectId('6193944674c5e6d7a8ac4c33'), ObjectId('6193944774c5e6d7a8ac4c3e'), ObjectId('6193944774c5e6d7a8ac4c4e')], 'DBPIA': [ObjectId('6193944705488d4887ad9217'), ObjectId('6193945605488d4887ad938b'), ObjectId('6193945d05488d4887ad948f')]}
{'name': '김성민', 'NTIS': [ObjectId('6193944674c5e6d7a8ac4c14'), ObjectId('6193944774c5e6d7a8ac4c66')], 'DBPIA': [ObjectId('6193944405488d4887ad9205'), ObjectId('6193945605488d4887ad93c7')]}
{'name': '홍순구', 'NTIS': [ObjectId('6193944674c5e6d7a8ac4c12')], 'DBPIA': [ObjectId('6193945305488d4887ad933e'), Obje